now you need to fill this with all the chunks techniques 12-13

start with 2 then 3 then 4: compare which ones has the best results
remove the input id things so you get the correct  thing for the correct scores 13:30-16:30


generate the augmented data as well so you can also compare: (the code is there, you just need to generate that and repeat the last part for a different dataset

get s imple explaination for the maths of roberta from chatgp
t before meeting 10:00-10:30

list all the relevant papers before meeting 11:30-1:30


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import numpy as np
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer


import subprocess


import sys

/home/users/mabdelaal/new/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Ensure the German SpaCy model is installed
try:
    nlp = spacy.load("de_core_news_sm")
except OSError:
    print("Downloading 'de_core_news_sm' model...")
    subprocess.run([sys.executable, "-m", "spacy", "download", "de_core_news_sm"], check=True)
    nlp = spacy.load("de_core_news_sm")



In [7]:
def extract_features(text):
    """
    Extracts linguistic and statistical features from text.
    """
    doc = nlp(text)
    
    # Part-of-speech tags (POS)
    pos_tags = [token.pos_ for token in doc]
    
    # Dependency relations
    dependencies = [token.dep_ for token in doc]
    
    # Sentence length
    sentence_length = len(doc)
    
    # Count of punctuation marks
    punctuation_count = sum(1 for token in doc if token.is_punct)
    
    return {
        "pos_tags": pos_tags,
        "dependencies": dependencies,
        "sentence_length": sentence_length,
        "punctuation_count": punctuation_count,
    }


In [8]:
def parse_rst_to_segments(rst_content):
    """
    Parses the RST content in XML format and extracts segments as a list of dictionaries.
    Each dictionary contains the text of a segment and extracted linguistic features.
    """
    segments = []
    root = ET.fromstring(rst_content)
    body = root.find("body")
    if body is not None:
        for segment in body.findall("segment"):
            text = segment.text.strip() if segment.text else ""
            if text:
                features = extract_features(text)
                segments.append({"text": text, **features})
    return segments

In [22]:
def prepare_segmentation_data_chunked(rst_files, chunk_size=4):
    """
    Prepares segmentation data in BIO format from RST files, including extracted features.
    """
    tokens_list, labels_list, pos_list, dep_list, lengths, punct_counts = [], [], [], [], [], []
    label_map = {"B-EDU": 0, "I-EDU": 1, "O": 2}

    for rst_file in tqdm(rst_files):
        try:
            with open(rst_file, 'r') as file:
                rst_content = file.read()
                segments = parse_rst_to_segments(rst_content)

                chunk_tokens, chunk_labels, chunk_pos, chunk_dep, chunk_lengths, chunk_punct = [], [], [], [], [], []
                for i, segment in enumerate(segments):
                    tokens = tokenizer.tokenize(segment['text'])
                    labels = [label_map['B-EDU']] + [label_map['I-EDU']] * (len(tokens) - 1)

                    chunk_tokens.extend(tokens)
                    chunk_labels.extend(labels)
                    chunk_pos.extend(segment['pos_tags'])
                    chunk_dep.extend(segment['dependencies'])
                    chunk_lengths.append(segment['sentence_length'])
                    chunk_punct.append(segment['punctuation_count'])

                    if (i + 1) % chunk_size == 0 or i == len(segments) - 1:
                        tokens_list.append(chunk_tokens)
                        labels_list.append(chunk_labels)
                        pos_list.append(chunk_pos)
                        dep_list.append(chunk_dep)
                        lengths.append(chunk_lengths)
                        punct_counts.append(chunk_punct)
                        chunk_tokens, chunk_labels, chunk_pos, chunk_dep, chunk_lengths, chunk_punct = [], [], [], [], [], []

        except Exception as e:
            print(f"Error processing {rst_file}: {e}")

    return {"tokens": tokens_list, "labels": labels_list, "pos_tags": pos_list, "dependencies": dep_list, "sentence_length": lengths, "punctuation_count": punct_counts}

def prepare_segmentation_data_from_folder_chunked(folder_path):
    """
    Prepares segmentation data from all RST files in a folder, avoiding hidden files and non-regular files.
    """
    rst_files = []
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        # Skip hidden files and non-regular files (e.g., directories, symlinks)
        if file_name.startswith(".") or not os.path.isfile(file_path):
            continue
        
        # Only process .rs3 files
        if file_name.endswith(".rs3"):
            rst_files.append(file_path)
    
    return prepare_segmentation_data_chunked(rst_files)

In [23]:

model_name = "xlm-roberta-base"  # or "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_function_full(examples):
    """
    Tokenizes full-text inputs while maintaining correct segmentation label alignment.
    """
    max_length = 512
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        is_split_into_words=True,
        max_length=max_length,
    )

    all_labels = []
    for i, labels in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = [-100] * len(word_ids)
        
        previous_word_id = None
        label_index = 0

        for idx, word_id in enumerate(word_ids):
            if word_id is None:
                continue
            
            if previous_word_id is None or word_id != previous_word_id:
                if label_index < len(labels):
                    aligned_labels[idx] = labels[label_index]
                label_index += 1
            else:
                if label_index - 1 < len(labels):
                    aligned_labels[idx] = labels[label_index - 1]

            previous_word_id = word_id

        all_labels.append(aligned_labels[:max_length])
    
    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs


In [24]:
rst_folder_path = "data/pcc-main/rs3_no_aug/pcc_augmented"
segmentation_data = prepare_segmentation_data_from_folder_chunked(rst_folder_path)
segmentation_dataset = Dataset.from_dict(segmentation_data)
tokenized_dataset = segmentation_dataset.map(tokenize_function_full, batched=True)


Map: 100%|██████████| 11661/11661 [01:15<00:00, 154.30 examples/s]


In [25]:
print(tokenized_dataset['labels'][0])


[-100, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 

In [26]:
# Train-test split

train_test_split_data_two = tokenized_dataset.train_test_split(test_size=0.2)

seg_dataset_chunked = DatasetDict({"train": train_test_split_data_two["train"], "test": train_test_split_data_two["test"]})



In [27]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)  # Get the predicted class for each token
    
    # Flatten the arrays to compute metrics at the token level
    labels_flat = labels.flatten()
    preds_flat = preds.flatten()
    
    # Filter out ignored index (-100) if applicable
    mask = labels_flat != -100
    labels_filtered = labels_flat[mask]
    preds_filtered = preds_flat[mask]
    
    # Compute metrics
    precision, recall, f1, _ = precision_recall_fscore_support(labels_filtered, preds_filtered, average="weighted")
    accuracy = accuracy_score(labels_filtered, preds_filtered)
    
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [28]:


# Custom Trainer for Weighted Loss
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        outputs = model(**inputs)
        logits = outputs.logits
        labels = inputs["labels"]

        # Define class weights (adjust if needed)
        loss_weights = torch.tensor([2.0, 1.0, 1.0]).to(logits.device)

        # Compute weighted loss
        loss_function = torch.nn.CrossEntropyLoss(weight=loss_weights, ignore_index=-100)
        loss = loss_function(logits.view(-1, logits.size(-1)), labels.view(-1))

        return (loss, outputs) if return_outputs else loss
        
model_name = "xlm-roberta-base"
segmentation_model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=3, device_map="auto",)  # B-EDU, I-EDU, O
segmentation_model.gradient_checkpointing_enable()


# Step 6: Training Arguments
seg_training_args = TrainingArguments(
    output_dir="./Models/segmentation_model_base_4_chunks_features_aug",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,  # Reduced batch size
    per_device_eval_batch_size=2,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="epoch",
    gradient_accumulation_steps=4,  # Gradient accumulation
    fp16=True,  # Mixed precision
)

# Step 7: Trainer Setup
seg_trainer = CustomTrainer(
    model=segmentation_model,
    args=seg_training_args,
    train_dataset=seg_dataset_chunked["train"],
    eval_dataset=seg_dataset_chunked["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)



# Step 8: Training
seg_trainer.train()

# Step 9: Evaluation
seg_results = seg_trainer.evaluate()

print("Segmentation Evaluation:", seg_results)



# Save Model
seg_trainer.model.save_pretrained("./Models/segmentation_model_base_4_chunks_features_aug")


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/users/mabdelaal/new/venv/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_23297/3059912100.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  seg_trainer = CustomTrainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.179000,0.150920,0.958779,0.957393,0.958779,0.958010
2,0.138000,0.130395,0.964998,0.966153,0.964998,0.965521
3,0.106100,0.106671,0.971913,0.972498,0.971913,0.972182
4,0.080900,0.101614,0.975612,0.975899,0.975612,0.975747
5,0.069000,0.102809,0.975935,0.976272,0.975935,0.976092


Segmentation Evaluation: {'eval_loss': 0.10280866175889969, 'eval_accuracy': 0.9759350224060669, 'eval_precision': 0.9762721875513638, 'eval_recall': 0.9759350224060669, 'eval_f1': 0.9760922578741269, 'eval_runtime': 207.9148, 'eval_samples_per_second': 11.221, 'eval_steps_per_second': 5.613, 'epoch': 5.0}


In [17]:


torch.cuda.empty_cache()

In [29]:
def prepare_segmentation_test_data_chunked(folder_path, chunk_size=4):
    """
    Prepare test data for the segmentation model from all files in a folder,
    ensuring the format is consistent with the training labels.
    Includes extracted linguistic features.
    """
    tokens_list, labels_list, features_list = [], [], []
    label_map = {"B-EDU": 0, "I-EDU": 1, "O": 2}
    all_sentences = []

    for file_name in os.listdir(folder_path):
        if file_name.startswith("."):
            continue
        
        file_path = os.path.join(folder_path, file_name)
        if not os.path.isfile(file_path):
            continue
        
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    parts = line.strip().split(" ")
                    text = " ".join(parts[1:-1]).strip("[]")
                    all_sentences.append(text)
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

    chunk_tokens, chunk_labels, chunk_features = [], [], []
    for i, sentence in enumerate(all_sentences):
        tokens = sentence.split()
        labels = [label_map['B-EDU']] + [label_map['I-EDU']] * (len(tokens) - 1)
        features = extract_features(sentence)
        
        chunk_tokens.extend(tokens)
        chunk_labels.extend(labels)
        chunk_features.append(features)

        if (i + 1) % chunk_size == 0 or i == len(all_sentences) - 1:
            tokens_list.append(chunk_tokens)
            labels_list.append(chunk_labels)
            features_list.append(chunk_features)
            chunk_tokens, chunk_labels, chunk_features = [], [], []

    return {"tokens": tokens_list, "labels": labels_list, "features": features_list}

In [30]:
# Example usage
folder_path = "./data/Essays_dataset"  # Replace with your dataset folder path
test_data_chunked = prepare_segmentation_test_data_chunked(folder_path)  # Adjust chunk size as needed
tokenized_test_dataset = Dataset.from_dict(test_data_chunked)

# Apply tokenization function correctly
tokenized_test_dataset_chunked = tokenized_test_dataset.map(tokenize_function_full, batched=True)

Map: 100%|██████████| 160/160 [00:00<00:00, 160.84 examples/s]


In [31]:
model = AutoModelForTokenClassification.from_pretrained("./Models/segmentation_model_base_4_chunks_features_aug")

model.eval()

XLMRobertaForTokenClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768

In [21]:
# Predict on Test Data
# Tokenize the test data



# Predict on Test Data
predictions = []
no_in = []
with torch.no_grad():
    for input_ids, attention_mask in tqdm(zip(tokenized_test_dataset_chunked["input_ids"], tokenized_test_dataset_chunked["attention_mask"])):
        input_ids = torch.tensor([input_ids])
        attention_mask = torch.tensor([attention_mask])

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1).squeeze().tolist()

        # Filter out padding tokens
        filtered_preds = [
            p for p, mask in zip(preds, attention_mask.squeeze().tolist()) if mask == 1
        ]
        no_in_pred = [
            p for p, mask,in_id in zip(preds, attention_mask.squeeze().tolist(),input_ids.squeeze().tolist()) if mask == 1 and in_id not in [0,2,1]
        ]
        predictions.append(filtered_preds)
        no_in.append(no_in_pred)

# Map Predictions Back to Labels
inverse_label_map = {0: "B-EDU", 1: "I-EDU", 2: "O"}
predicted_labels = []
for pred in predictions:
    predicted_labels.append([inverse_label_map[label] for label in pred])

# Flatten labels for evaluation
true_labels_flat = []
predicted_labels_flat = []
for true, pred, attention_mask in zip(tokenized_test_dataset_chunked["labels"], predictions, tokenized_test_dataset_chunked["attention_mask"]):
    for t, p, mask in zip(true, pred, attention_mask):
        if mask == 1 and t != -100:  # Exclude padding and special tokens
            true_labels_flat.append(t)
            predicted_labels_flat.append(p)

# Compute Metrics
precision, recall, f1, _ = precision_recall_fscore_support(true_labels_flat, predicted_labels_flat, average="weighted")
accuracy = accuracy_score(true_labels_flat, predicted_labels_flat)


print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")




213it [22:10,  6.25s/it]


Precision: 0.9443460061251111
Recall: 0.946426411899867
F1 Score: 0.9452027281481584
Accuracy: 0.946426411899867


In [23]:
problems = []
for idx, (t , p) in enumerate(zip(tokenized_test_dataset_chunked["labels"], predictions)):

    if t != p:
        problems.append((idx, (t,p)))

In [24]:
print(problems[2][1][0])
print(problems[2][1][1])
print(tokenized_test_dataset_chunked["labels"][2])
print(predictions[2])
print(tokenized_test_dataset_chunked["tokens"][2])


[-100, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 

In [ ]:
len(problems)

In [44]:
# Example German test text
test_text = """
Die künstliche Intelligenz hat in den letzten Jahren erhebliche Fortschritte gemacht. Einerseits bietet sie zahlreiche Vorteile, andererseits gibt es auch ethische Bedenken.
Ein Hauptargument für KI ist die Effizienzsteigerung. Maschinen können Aufgaben schneller und präziser erledigen als Menschen.
Jedoch gibt es auch Gegenargumente: Viele befürchten den Verlust von Arbeitsplätzen.
"""

# Tokenize the text
encoded_text = tokenizer(test_text, truncation=True, padding=True, return_tensors="pt")


In [46]:
test_text = """
Die künstliche Intelligenz verändert unsere Welt sie beeinflusst wie wir arbeiten wie wir 
lernen wie wir miteinander kommunizieren Viele Experten glauben dass KI den Arbeitsmarkt 
revolutionieren wird sie könnte ineffiziente Prozesse verbessern und Produktivität steigern. 
Aber es gibt auch Sorgen dass viele Jobs ersetzt werden und Menschen langfristig arbeitslos 
sein könnten die soziale Ungleichheit könnte dadurch wachsen Einige Regierungen haben bereits
Maßnahmen ergriffen um neue Bildungsprogramme zu fördern damit Menschen sich besser anpassen 
können an den technologischen Wandel. KI bringt sowohl Chancen als auch 
Risiken es hängt davon ab wie wir sie einsetzen und ob wir als Gesellschaft kluge 
Entscheidungen treffen"""
# Tokenize the text
encoded_text = tokenizer(test_text, truncation=True, padding=True, return_tensors="pt")

In [47]:
# Run inference
with torch.no_grad():
    outputs = model(**encoded_text)

# Get predictions
predictions = torch.argmax(outputs.logits, dim=-1)

# Convert back to labels
label_map = {0: "B-EDU", 1: "I-EDU", 2: "O"}  # Adjust this based on your training labels
predicted_labels = [label_map[label.item()] for label in predictions[0]]

# Print segmented output
tokens = tokenizer.convert_ids_to_tokens(encoded_text["input_ids"][0])
for token, label in zip(tokens, predicted_labels):
    print(f"{token}: {label}")

<s>: I-EDU
▁Die: B-EDU
▁kü: I-EDU
n: I-EDU
st: I-EDU
liche: I-EDU
▁Intel: I-EDU
ligen: I-EDU
z: I-EDU
▁verändert: I-EDU
▁unsere: I-EDU
▁Welt: I-EDU
▁sie: I-EDU
▁be: I-EDU
ein: I-EDU
fluss: I-EDU
t: I-EDU
▁wie: I-EDU
▁wir: I-EDU
▁arbeiten: I-EDU
▁wie: I-EDU
▁wir: I-EDU
▁lernen: I-EDU
▁wie: I-EDU
▁wir: I-EDU
▁miteinander: I-EDU
▁kommun: I-EDU
i: I-EDU
zieren: I-EDU
▁Viele: I-EDU
▁Experten: I-EDU
▁glauben: I-EDU
▁dass: I-EDU
▁KI: I-EDU
▁den: I-EDU
▁Arbeits: I-EDU
markt: I-EDU
▁revolution: I-EDU
ieren: I-EDU
▁wird: I-EDU
▁sie: I-EDU
▁könnte: I-EDU
▁in: I-EDU
effizient: I-EDU
e: I-EDU
▁Prozess: I-EDU
e: I-EDU
▁verbessern: I-EDU
▁und: I-EDU
▁Produkt: I-EDU
iv: I-EDU
ität: I-EDU
▁: I-EDU
steiger: I-EDU
n: I-EDU
.: I-EDU
▁Aber: I-EDU
▁es: B-EDU
▁gibt: I-EDU
▁auch: I-EDU
▁Sor: I-EDU
gen: I-EDU
▁dass: I-EDU
▁viele: I-EDU
▁Jobs: I-EDU
▁er: I-EDU
setzt: I-EDU
▁werden: I-EDU
▁und: I-EDU
▁Menschen: I-EDU
▁langfristig: I-EDU
▁: I-EDU
arbeit: I-EDU
s: I-EDU
los: I-EDU
▁sein: I-EDU
▁könnten: I-EDU
▁die